In [1]:
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, UpSampling2D, Conv2D
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras import backend as K
import keras

import numpy as np
import matplotlib.pyplot as plt

from os import listdir
from os import system
import os
import random

import imageio

Using TensorFlow backend.


In [2]:
def fixed_generator(generator):
    for batch in generator:
        yield (batch, batch)


In [3]:
img_width, img_height = 16, 16

nb_epoch = 50
batch_size = 32

In [4]:
base_dir = '/home/niaki/Code/ImageNet/tiny-imagenet-200'

train_data_dir      = base_dir + '/tiny_train16'
validation_data_dir = base_dir + '/tiny_validation16'
test_data_dir       = base_dir + '/tiny_test16'


nb_train_samples = 0
train_folders = os.listdir(train_data_dir)
train_folders.sort()
for folder in train_folders:
    nb_train_samples += len(os.listdir(train_data_dir + '/' + folder))
print(nb_train_samples)

nb_validation_samples = 0
validation_folders = os.listdir(validation_data_dir)
validation_folders.sort()
for folder in validation_folders:
    nb_validation_samples += len(os.listdir(validation_data_dir + '/' + folder))
print(nb_validation_samples)

157086
3932


In [7]:
input_shape = (img_width, img_height, 3)
input_img = Input(shape=input_shape)


W0808 17:03:30.658493 140588966696768 deprecation_wrapper.py:119] From /scratch/tensorflow/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



In [19]:
x = Conv2D(32, (3, 3), activation="relu", padding="same")(input_img)
x = Conv2D(32, (3, 3), activation="relu", padding="same", dilation_rate=2)(x)
x = Conv2D(32, (3, 3), activation="relu", padding="same", dilation_rate=4)(x)

model = Model(input_img, x)



In [20]:
for i in range(len(model.layers)):
    print(model.layers[i].output_shape)


(None, 16, 16, 3)
(None, 16, 16, 32)
(None, 16, 16, 32)
(None, 16, 16, 32)


In [ ]:


x = Conv2D(32, (3, 3), activation="relu", padding="same")(input_img)
x = MaxPooling2D((2, 2), padding="same")(x)
x = Conv2D(32, (3, 3), activation="relu", padding="same")(x)
x = MaxPooling2D((2, 2), padding="same")(x)
x = Conv2D(32, (3, 3), activation="relu", padding="same")(x)
x = MaxPooling2D((2, 2), padding="same")(x)
x = Conv2D(16, (3, 3), activation="relu", padding="same")(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)


x = Conv2D(16, (3, 3), activation="relu", padding="same")(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation="relu", padding="same")(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation="relu", padding="same")(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation="relu", padding="same")(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation="sigmoid", padding="same")(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

autoencoder.summary()